### Read multiple JSON files 

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [0]:
qualifying_schema = StructType([StructField("qualifyId", IntegerType(), False), 
                              StructField("raceId", IntegerType(), True),
                              StructField("driverId", IntegerType(), True),
                              StructField("constructorId", IntegerType(), True),
                              StructField("number", IntegerType(), True),
                              StructField("position", IntegerType(), True),
                              StructField("q1", StringType(), True),
                              StructField("q2", StringType(), True),
                              StructField("q3", StringType(), True),
                            ])


qualifying_df = spark.read \
    .schema(qualifying_schema) \
    .option("multiline", True) \
    .json(f"{RAW_FOLDER_PATH}/qualifying")   


In [0]:
qualifying_df.printSchema()

In [0]:
display(qualifying_df)

### rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, date_trunc, from_utc_timestamp

In [0]:
final_qualifying_df = add_ingestion_date(qualifying_df.withColumnRenamed("qualifyId", "qualify_id") \
                                .withColumnRenamed("driverId", "driver_id") \
                                .withColumnRenamed("raceId", "race_id") \
                                .withColumnRenamed("constructorId", "constructor_id"))



display(final_qualifying_df)

### write to parquet file

In [0]:
final_qualifying_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.qualifying")



In [0]:
spark.read.parquet(f"{PROCESSED_FOLDER_PATH}/qualifying").display()

In [0]:
dbutils.notebook.exit("Success")